In [1]:
import numpy as np 
from PIL import Image
import os
import subprocess
from datetime import datetime


def open_paint(image_path):
    if os.name == 'nt':  # Checks if the system is Windows
        subprocess.run(["mspaint", image_path], check=True)
    else:
        raise OSError("Microsoft Paint is not available on this system.")

def edit_heatmap_in_paint(directory, image_prefix="custom", size=200):
    image_path = os.path.join(directory, f"{image_prefix}.png")
    if not os.path.exists(image_path):
        heatmap = np.ones((size, size), dtype=np.uint8) * 255
        Image.fromarray(heatmap).save(image_path)

    open_paint(image_path)

def list_heatmap_files(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
    return [os.path.splitext(f)[0] for f in os.listdir(directory) if f.endswith(".png")]

def load_heatmap(directory, image_prefix, mode, inverted):
    image_path = os.path.join(directory, f"{image_prefix}.png")
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"The heatmap '{image_path}' does not exist. Use 'create_or_edit_heatmap' to create it.")

    image = Image.open(image_path)

    if mode == "L":
        # Corrects the background in case of transparent pixels
        image = Image.alpha_composite(
            Image.new("RGBA", image.size, (255, 255, 255, 255)),
            image.convert("RGBA")
        ).convert("L")
    else:
        image = image.convert(mode)

    heatmap = np.array(image)
    heatmap = np.flipud(heatmap)  # Vertically flip to match the desired orientation

    if inverted:
        heatmap = 255 - heatmap

    return heatmap

def save_generated_figure(directory="results", filename_prefix="voronoi"):
    global stored_figure
    if stored_figure is None:
        print("No figure has been generated yet.")
        return

    os.makedirs(directory, exist_ok=True)

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    file_path = os.path.join(directory, f"{filename_prefix}_{timestamp}.png")

    stored_figure.savefig(file_path, bbox_inches='tight', pad_inches=0)
    print(f"Figure saved at: {file_path}")


In [2]:
import matplotlib.pyplot as plt
from matplotlib import patches, path
from matplotlib.colors import to_rgb, to_hex, ListedColormap, LinearSegmentedColormap
from scipy.spatial import Voronoi, voronoi_plot_2d

def shrink(polygon, pad):
    center = np.mean(polygon, axis=0)
    resized = np.zeros_like(polygon)
    for ii, point in enumerate(polygon):
        vector = point - center
        unit_vector = vector / np.linalg.norm(vector)
        resized[ii] = point - pad * unit_vector
    return resized


class RoundedPolygon(patches.PathPatch):
    def __init__(self, xy, pad, **kwargs):
        p = path.Path(*self.__round(xy=xy, pad=pad))
        super().__init__(path=p, **kwargs)

    def __round(self, xy, pad):
        n = len(xy)

        for i in range(0, n):

            x0, x1, x2 = np.atleast_1d(xy[i - 1], xy[i], xy[(i + 1) % n])

            d01, d12 = x1 - x0, x2 - x1
            l01, l12 = np.linalg.norm(d01), np.linalg.norm(d12)
            u01, u12 = d01 / l01, d12 / l12

            x00 = x0 + min(pad, 0.5 * l01) * u01
            x01 = x1 - min(pad, 0.5 * l01) * u01
            x10 = x1 + min(pad, 0.5 * l12) * u12
            x11 = x2 - min(pad, 0.5 * l12) * u12

            if i == 0:
                verts = [x00, x01, x1, x10]
            else:
                verts += [x01, x1, x10]

        codes = [path.Path.MOVETO] + n*[path.Path.LINETO, path.Path.CURVE3, path.Path.CURVE3]

        verts[0] = verts[-1]

        return np.atleast_1d(verts, codes)

def generate_points_with_heatmap(n, max_x, max_y, heatmap, min_mask, max_mask):
    if heatmap is None:
        raise ValueError("Invalid or missing heatmap")
    
    base_points = np.c_[np.random.uniform(0, max_x, size=n),
                        np.random.uniform(0, max_y, size=n)]

    min_gray = np.min(heatmap)
    max_gray = np.max(heatmap)

    if min_gray == max_gray:
        return base_points

    normalized_heatmap = (heatmap - min_gray) / (max_gray - min_gray)

    mask_multiplier = min_mask + (1 - normalized_heatmap) * (max_mask - min_mask)

    total_additional_density = mask_multiplier - 1
    total_additional_density[total_additional_density < 0] = 0  # No negative density

    additional_points_count = int(np.sum(total_additional_density) * (n / heatmap.size))

    # Generate additional points influenced by the heatmap
    probabilities = total_additional_density / np.sum(total_additional_density)
    probabilities_flat = probabilities.flatten()
    indices = np.random.choice(len(probabilities_flat), size=additional_points_count, p=probabilities_flat)
    y_indices, x_indices = np.unravel_index(indices, probabilities.shape)

    # Convert indices to real-world coordinates
    x = x_indices * (max_x / heatmap.shape[1])
    y = y_indices * (max_y / heatmap.shape[0])
    additional_points = np.c_[x, y]

    # Combine base points with heatmap-influenced points
    points = np.vstack([base_points, additional_points])
    return points

def generate_points(seed, n, max_x, max_y, distribution, min_mask, max_mask, inverted):
    np.random.seed(seed)

    if distribution == 'uniform':
        points = np.c_[np.random.uniform(0, max_x, size=n),
                       np.random.uniform(0, max_y, size=n)]
    elif distribution == 'normal':
        points = np.random.normal(loc=[max_x / 2, max_y / 2],
                                   scale=[max_x / 4, max_y / 4],
                                   size=(n, 2))
    else:
        heatmap = load_heatmap("densities", distribution, "L", inverted)
        points = generate_points_with_heatmap(n, max_x, max_y, heatmap, min_mask, max_mask)

    # Add distant points to avoid artefacts
    points = np.append(points, [[2 * max_x, 2 * max_y],
                                [-max_x, 2 * max_y],
                                [2 * max_x, -max_y],
                                [-max_x, -max_y]], axis=0)
    return points

def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip('#')
    return tuple(int(hex_color[i:i + 2], 16) / 255.0 for i in (0, 2, 4))

def apply_mask_to_color(base_color, mask_color, intensity):
    base = np.array(base_color)
    mask_rgb = np.array(mask_color[:3]) / 255.0
    alpha = mask_color[3] / 255.0

    blended = base + alpha * intensity * (mask_rgb - base)

    return tuple(np.clip(blended, 0, 1))

def get_texture_from_mode(mode, directory):
    if mode == "None":
        return None
    
    return load_heatmap(directory, mode, "RGBA", False)

def apply_mask_to_voronoi(base_color, max_x, max_y, resized, color_heatmap, intensity):
    if color_heatmap is not None:
        center = np.mean(resized, axis=0)
        x_idx = int(center[0] / max_x * color_heatmap.shape[1])
        y_idx = int(center[1] / max_y * color_heatmap.shape[0])
        
        x_idx = np.clip(x_idx, 0, color_heatmap.shape[1] - 1)
        y_idx = np.clip(y_idx, 0, color_heatmap.shape[0] - 1)

        mask_color = color_heatmap[y_idx, x_idx]
        return apply_mask_to_color(base_color, mask_color, intensity)

    return base_color

stored_figure = None
def plot_voronoi_tesselation(seed, round_coeff,
                             n, max_x, max_y, background_color, shrink_factor,
                             base_color, color_intensity_min, color_intensity_max,
                             density_mode, min_density, max_density, inverted,
                             texture_mode, texture_intensity,
                             shadows_mode, shadows_intensity):
    global stored_figure
    
    points=generate_points(seed, n, max_x, max_y, density_mode, min_density, max_density, inverted)
    vor = Voronoi(points)
    fig, ax = plt.subplots(figsize=(max_x, max_y))

    base_color_rgb = hex_to_rgb(base_color)
    
    texture_heatmap = get_texture_from_mode(texture_mode, "textures")
    shadows_heatmap = get_texture_from_mode(shadows_mode, "shadows")

    np.random.seed(seed + 1)

    for region in vor.regions:
        if region and -1 not in region:
            polygon = np.array([vor.vertices[i] for i in region])
            resized = shrink(polygon, shrink_factor)

            color_intensity = np.random.uniform(color_intensity_min, color_intensity_max)
            final_color = tuple(np.array(base_color_rgb) * color_intensity)
            final_color = apply_mask_to_voronoi(final_color, max_x, max_y, resized, texture_heatmap, texture_intensity)
            final_color = apply_mask_to_voronoi(final_color, max_x, max_y, resized, shadows_heatmap, shadows_intensity)

            if round_coeff <= 0.0:
                polygon = patches.Polygon(resized, color=final_color)
            else:
                polygon = RoundedPolygon(xy=resized, pad=round_coeff, color=final_color)
            ax.add_patch(polygon)

    ax.axis([0, max_x, 0, max_y])
    ax.axis('off')
    ax.set_facecolor(background_color)
    ax.add_artist(ax.patch)
    ax.patch.set_zorder(-1)
    plt.show()

    stored_figure = fig


In [4]:
from ipywidgets import interact, IntSlider, FloatSlider, ColorPicker, Checkbox, Dropdown, Button, HBox, VBox, Label, Output, Layout
from IPython.display import display

densities = ["uniform", "normal", "custom"] + list_heatmap_files("densities")
base_colors = ["None", "custom"]
textures = base_colors + list_heatmap_files("textures")
shadows = base_colors + list_heatmap_files("shadows")

# Widgets pour les paramètres
seed_slider = IntSlider(value=42, min=-1, max=100, step=1, description='Seed')
round_coeff_slider = FloatSlider(value=0.2, min=0.0, max=5.0, step=0.02, description='Round Coeff')
n_slider = IntSlider(value=150, min=10, max=1000, step=10, description='Points')
max_x_slider = FloatSlider(value=25, min=5, max=50, step=1, description='Width')
max_y_slider = FloatSlider(value=10, min=5, max=50, step=1, description='Height')
background_color_picker = ColorPicker(value="#000000", description="Background")
shrink_factor_slider = FloatSlider(value=0.04, min=0.0, max=0.5, step=0.02, description='Shrink')
base_color_picker = ColorPicker(value="#3477db", description="Base Color")
color_intensity_min_slider = FloatSlider(value=0.5, min=0.0, max=1.0, step=0.01, description='Intensity Min')
color_intensity_max_slider = FloatSlider(value=1.0, min=0.0, max=1.0, step=0.01, description='Intensity Max')

# Dropdowns et sliders supplémentaires
density_mode_dropdown = Dropdown(options=densities, value="uniform", description="Density")
min_density_slider = FloatSlider(value=0.01, min=0.01, max=1.0, step=0.01, description='Min Density')
max_density_slider = FloatSlider(value=10.0, min=1.0, max=50.0, step=0.1, description='Max Density')
inverted_checkbox = Checkbox(value=False, description='Inverted')
texture_mode_dropdown = Dropdown(options=textures, value="None", description="Texture")
texture_intensity_slider = FloatSlider(value=0.2, min=0.0, max=1.0, step=0.01, description='Texture Intensity')
shadows_mode_dropdown = Dropdown(options=shadows, value="None", description="Shadows")
shadows_intensity_slider = FloatSlider(value=0.2, min=0.0, max=1.0, step=0.01, description='Shadows Intensity')

# Organisation des paramètres en colonnes
general_settings = VBox([Label("General Settings"), seed_slider, n_slider, max_x_slider, max_y_slider, shrink_factor_slider, round_coeff_slider])
color_settings = VBox([Label("Color Settings"), background_color_picker, base_color_picker, color_intensity_min_slider, color_intensity_max_slider,
                       texture_mode_dropdown, texture_intensity_slider, shadows_mode_dropdown, shadows_intensity_slider])
density_settings = VBox([Label("Density Settings"), density_mode_dropdown, min_density_slider, max_density_slider, inverted_checkbox])

# Widget Output pour afficher le rendu Voronoi
output_widget = Output()

# Boutons en bas
density_button = Button(description="Edit Density Heatmap")
texture_button = Button(description="Edit Texture")
shadows_button = Button(description="Edit Shadows")
save_button = Button(description="Save Voronoi")
buttons_row = HBox([density_button, texture_button, shadows_button, save_button])

parameters_columns = HBox(
    [VBox([general_settings]), VBox([color_settings]), VBox([density_settings])],
    layout=Layout(
        justify_content="space-between",  # Centre les éléments horizontalement
        align_items="flex-start",  # Ne centre pas verticalement
        width="100%"               # Prend toute la largeur disponible
    )
)

# Callback pour mettre à jour le rendu Voronoi
def update_voronoi(change=None):
    with output_widget:
        output_widget.clear_output(wait=True)  # Nettoyer le précédent rendu
        plot_voronoi_tesselation(
            seed=seed_slider.value,
            round_coeff=round_coeff_slider.value,
            n=n_slider.value,
            max_x=max_x_slider.value,
            max_y=max_y_slider.value,
            background_color=background_color_picker.value,
            shrink_factor=shrink_factor_slider.value,
            base_color=base_color_picker.value,
            color_intensity_min=color_intensity_min_slider.value,
            color_intensity_max=color_intensity_max_slider.value,
            density_mode=density_mode_dropdown.value,
            min_density=min_density_slider.value,
            max_density=max_density_slider.value,
            inverted=inverted_checkbox.value,
            texture_mode=texture_mode_dropdown.value,
            texture_intensity=texture_intensity_slider.value,
            shadows_mode=shadows_mode_dropdown.value,
            shadows_intensity=shadows_intensity_slider.value
        )

# Lier les callbacks aux widgets
for widget in [
    seed_slider, round_coeff_slider, n_slider, max_x_slider, max_y_slider,
    background_color_picker, shrink_factor_slider,
    base_color_picker, color_intensity_min_slider, color_intensity_max_slider,
    density_mode_dropdown, min_density_slider, max_density_slider, inverted_checkbox,
    texture_mode_dropdown, texture_intensity_slider,
    shadows_mode_dropdown, shadows_intensity_slider
]:
    widget.observe(update_voronoi, names='value')

# Effets des boutons
density_output = Output()
texture_output = Output()
shadows_output = Output()
save_output = Output()

@density_output.capture()
def on_density_button_click(_):
    edit_heatmap_in_paint("densities")
@texture_output.capture()
def on_texture_button_click(_):
    edit_heatmap_in_paint("textures")
@shadows_output.capture()
def on_shadows_button_click(_):
    edit_heatmap_in_paint("shadows")
@save_output.capture()
def on_save_button_click(_):
    save_generated_figure()

density_button.on_click(on_density_button_click)
texture_button.on_click(on_texture_button_click)
shadows_button.on_click(on_shadows_button_click)
save_button.on_click(on_save_button_click)

# Affichage final
layout = VBox([parameters_columns, output_widget, buttons_row])
display(layout)

# Mise à jour initiale
update_voronoi()